In [1]:
from ase.io import read, write, Trajectory
import numpy as np

In [2]:
def CoM(clus):
    """
    Support function to set the origin of the cluster at the centre of the mass
    """
    (cx, cy, cz) = clus.get_center_of_mass()
    new_xyz = []
    for i, a in enumerate(clus):
        x, y, z = a.position
        x -= cx
        y -= cy
        z -= cz
        new_xyz.append((x, y, z))
    clus.set_positions(new_xyz)
    return clus

In [3]:
def checkSimilar(clus1, clus2):

    """Check whether two clusters are similar or not by comparing their moments of inertia"""
    Inertia1 = clus1.get_moments_of_inertia()
    Inertia2 = clus2.get_moments_of_inertia()
    # print(Inertia1, Inertia2, 'diff: ', Inertia1-Inertia2)

    tol = 0.01
    if (
        Inertia1[0] * (1 - tol) <= Inertia2[0] <= Inertia1[0] * (1 + tol)
        and Inertia1[1] * (1 - tol) <= Inertia2[1] <= Inertia1[1] * (1 + tol)
        and Inertia1[2] * (1 - tol) <= Inertia2[2] <= Inertia1[2] * (1 + tol)
    ):
        similar = True
    else:
        similar = False

    return similar

In [4]:
gen = 25

In [5]:
num_check_similar = 0
diff_dist_abs_sum_list = []

non_similar_clus = []
similar_clus = []
non_similar_clus_ene_list = []
similar_clus_ene_list = []
non_similar_clus_diff_dist = []
similar_clus_diff_dist = []

n_traj1_list = []
n_traj2_list = []

for k in range(gen):
    file1  = 'mut_after_relax_gen'+str(k+1)+'.traj'
    file2  = 'vasp_mut_after_relax_gen'+str(k+1)+'.traj'

    traj1 = Trajectory(file1)
    traj2 = Trajectory(file2)
    
    n_traj1 = len(traj1)
    n_traj2 = len(traj2)
    n_traj1_list.append(n_traj1)
    n_traj2_list.append(n_traj2)
    
    for i in range(len(traj1)):
        num_check_similar = num_check_similar + 1
        clus1 = traj1[i]
        clus2 = traj2[i]
        
        ene1 = clus1.get_potential_energy()
        ene2 = clus2.get_potential_energy()
        diff_ene = ene1 - ene2
        
        dist1 = clus1.get_all_distances() 
        dist2 = clus2.get_all_distances()
        diff_dist = dist1 - dist2
        diff_dist_abs = np.absolute(diff_dist)
        diff_dist_avg = np.sum(diff_dist_abs) / len(diff_dist)**2
        
        inertia1 = clus1.get_moments_of_inertia()
        inertia2 = clus2.get_moments_of_inertia()
        diff_inertia = inertia1 - inertia2
              
        if checkSimilar(clus1, clus2) == True:
            if diff_dist_avg > 0.02:
                if abs(diff_ene) > 0.02:
                    non_similar_clus.append((k,i))
                    non_similar_clus_ene_list.append(diff_ene)
                    non_similar_clus_diff_dist.append(diff_dist_avg)
                    print((k, i),diff_dist_avg, diff_ene, diff_inertia )
                else:
                    similar_clus.append((k,i))
                    similar_clus_ene_list.append(diff_ene)
                    similar_clus_diff_dist.append(diff_dist_avg)
                    #print((k, i),diff_dist_avg, diff_ene, diff_inertia )  
                    
            else:
                similar_clus.append((k,i))
                similar_clus_ene_list.append(diff_ene)
                similar_clus_diff_dist.append(diff_dist_avg)
                #print((k, i),diff_dist_avg, diff_ene, diff_inertia )  
                
        else:
            if diff_dist_avg < 0.02:
                if abs(diff_ene) < 0.02:
                    similar_clus.append((k,i))
                    similar_clus_ene_list.append(diff_ene)
                    similar_clus_diff_dist.append(diff_dist_avg)
                    #print((k, i),diff_dist_avg, diff_ene, diff_inertia )
                else: 
                    non_similar_clus.append((k,i))
                    non_similar_clus_ene_list.append(diff_ene)
                    non_similar_clus_diff_dist.append(diff_dist_avg)
                    print((k, i),diff_dist_avg, diff_ene, diff_inertia )
            else:
                non_similar_clus.append((k,i))
                non_similar_clus_ene_list.append(diff_ene)
                non_similar_clus_diff_dist.append(diff_dist_avg)
                #print((k, i),diff_dist_avg, diff_ene, diff_inertia )

print(len(non_similar_clus))
print(len(similar_clus))
print(num_check_similar)

#print(sorted(non_similar_clus_ene_list))
#print(non_similar_clus)
#print(sorted(non_similar_clus_ene_list))
#print(sorted(non_similar_clus_diff_dist))

print(n_traj1_list)
print(n_traj2_list)

230
18
248
[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 9, 10, 10, 10, 10, 10, 10, 10, 10, 9]
[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 9]


In [7]:
filename="clus_Ni6Pd4.log"

parent_calls_list = []
with open(filename, 'r') as fh:
    for line in fh:
        if line.startswith("Parent"):
            line = line.strip()
            words = line.split(':')
            words[1] = words[1].lstrip()
            parent_calls_list.append(words[1])
#print(parent_calls_list)
#print((parent_calls_list[0]))

item_split = []
for item in parent_calls_list:
    print(item)
    item_split.append(item.split(","))
#print(item_split)

#print(item_split[0])


[37, 7, 27, 67, 21, 33, 13, 30, 47, 11]
[28, 40, 11, 31, 24, 27, 44, 25, 25, 42]
[23, 18, 13, 38, 12, 30, 35, 41, 33, 21]
[33, 28, 18, 49, 30, 30, 17, 27, 24, 22]
[21, 26, 30, 34, 39, 29, 32, 34, 27, 34]
[19, 22, 24, 47, 28, 28, 52, 54, 21, 24]
[29, 29, 14, 12, 21, 35, 22, 42, 24, 33]
[40, 29, 37, 33, 26, 22, 26, 33, 46, 21]
[27, 15, 43, 28, 31, 35, 32, 24, 20, 32]
[26, 36, 23, 33, 41, 21, 21, 28, 33, 19]
[29, 30, 30, 19, 18, 41, 32, 29, 31, 13]
[30, 21, 30, 31, 24, 41, 34, 26, 36, 23]
[39, 17, 23, 30, 43, 30, 16, 29, 32, 24]
[31, 37, 27, 38, 18, 26, 6, 31, 32, 23]
[4, 39, 30, 34, 28, 34, 35, 36, 52, 27]
[10, 30, 28, 23, 39, 28, 38, 37, 21, 30]
[30, 24, 33, 41, 10, 26, 23, 35, 48, 27]
[32, 36, 22, 28, 36, 28, 6, 27, 35]
[22, 24, 37, 39, 26, 59, 22, 23, 20, 11]
[19, 34, 31, 18, 9, 30, 31, 35, 29, 31]
[21, 39, 24, 49, 28, 31, 29, 11, 38, 47]
[32, 32, 28, 36, 30, 25, 37, 16, 35, 13]
[7, 52, 24, 37, 25, 42, 21, 4, 19, 24]
[26, 32, 24, 23, 18, 26, 25, 7, 17, 27]
[40, 41, 24, 33, 31, 27, 27,

In [8]:
new_vasp_calls = []
len_new_vasp_calls = []
for i in range(len(item_split)):
    k=len(item_split[i])
    new_int_list = []
    
    a = list(item_split[i][0].lstrip())
    del a[0] 
    a = int(''.join(a))
    new_int_list.append(a)
    #print(a)
    for k in range(1,(len(item_split[i])-1)):
        b = int(item_split[i][k])
        new_int_list.append(b)
        #print(b)  
    c = list(item_split[i][-1].lstrip())
    del c[-1]
    c = int(''.join(c))
    new_int_list.append(c)
    #print(c)
    #print(new_int_list)
    len_new_list = len(new_int_list)
    new_vasp_calls.append(new_int_list)
    len_new_vasp_calls.append(len_new_list)
print(new_vasp_calls)
print(len(new_vasp_calls))
print(len_new_vasp_calls)
print(sum(len_new_vasp_calls))

[[37, 7, 27, 67, 21, 33, 13, 30, 47, 11], [28, 40, 11, 31, 24, 27, 44, 25, 25, 42], [23, 18, 13, 38, 12, 30, 35, 41, 33, 21], [33, 28, 18, 49, 30, 30, 17, 27, 24, 22], [21, 26, 30, 34, 39, 29, 32, 34, 27, 34], [19, 22, 24, 47, 28, 28, 52, 54, 21, 24], [29, 29, 14, 12, 21, 35, 22, 42, 24, 33], [40, 29, 37, 33, 26, 22, 26, 33, 46, 21], [27, 15, 43, 28, 31, 35, 32, 24, 20, 32], [26, 36, 23, 33, 41, 21, 21, 28, 33, 19], [29, 30, 30, 19, 18, 41, 32, 29, 31, 13], [30, 21, 30, 31, 24, 41, 34, 26, 36, 23], [39, 17, 23, 30, 43, 30, 16, 29, 32, 24], [31, 37, 27, 38, 18, 26, 6, 31, 32, 23], [4, 39, 30, 34, 28, 34, 35, 36, 52, 27], [10, 30, 28, 23, 39, 28, 38, 37, 21, 30], [30, 24, 33, 41, 10, 26, 23, 35, 48, 27], [32, 36, 22, 28, 36, 28, 6, 27, 35], [22, 24, 37, 39, 26, 59, 22, 23, 20, 11], [19, 34, 31, 18, 9, 30, 31, 35, 29, 31], [21, 39, 24, 49, 28, 31, 29, 11, 38, 47], [32, 32, 28, 36, 30, 25, 37, 16, 35, 13], [7, 52, 24, 37, 25, 42, 21, 4, 19, 24], [26, 32, 24, 23, 18, 26, 25, 7, 17, 27], [40

In [9]:
print(len(similar_clus))
print(similar_clus)

18
[(0, 9), (2, 1), (5, 0), (6, 1), (6, 2), (7, 9), (8, 2), (10, 3), (10, 4), (11, 3), (12, 3), (13, 0), (14, 8), (18, 2), (19, 0), (19, 7), (22, 2), (22, 9)]


In [10]:
file_vasp="results.log"

vasp_calls = []
vasp_calls_list = []
with open(file_vasp, 'r') as fh:
    total_images = []
    for line in fh:       
        if "VASP Calls" in line:
            line = line.strip()
            words = line.split(':')
            words[1] = int(words[1].lstrip())
            vasp_calls.append(words[1])
        if "Total images in the trajectory" in line:
            line = line.strip()
            img_num = line.split(':')
            img_num[1] = int(img_num[1].lstrip())
            total_images.append(img_num[1])

print(vasp_calls)
#print(len(vasp_calls))
#print(total_images)
#print(len(total_images))
#print(sum(total_images))
print('\n')

kk = 0
for i in total_images:
    traj_calls = []
    for j in range(kk, kk+i):
        traj_calls.append(vasp_calls[j])
    vasp_calls_list.append(traj_calls)
    kk = kk + i

print(vasp_calls_list)

[79, 19, 17, 18, 13, 21, 13, 69, 71, 19, 10, 17, 17, 41, 28, 23, 13, 41, 99, 22, 19, 15, 17, 36, 25, 13, 19, 13, 33, 13, 17, 23, 11, 63, 31, 18, 25, 19, 15, 17, 54, 81, 21, 21, 112, 15, 12, 17, 29, 27, 39, 21, 19, 74, 29, 52, 13, 37, 27, 17, 25, 21, 17, 12, 52, 52, 17, 19, 17, 54, 42, 40, 34, 15, 56, 19, 17, 11, 30, 15, 31, 68, 48, 27, 23, 19, 17, 15, 18, 23, 22, 26, 49, 55, 21, 19, 19, 52, 67, 34, 19, 9, 27, 47, 18, 13, 20, 33, 15, 29, 27, 17, 73, 58, 15, 69, 54, 90, 21, 19, 19, 21, 29, 15, 13, 17, 46, 17, 23, 15, 5, 17, 19, 79, 19, 17, 91, 63, 55, 38, 18, 27, 16, 55, 21, 35, 17, 111, 15, 49, 5, 71, 19, 19, 45, 15, 9, 13, 61, 78, 65, 11, 31, 18, 83, 29, 21, 33, 19, 17, 15, 55, 36, 19, 23, 19, 17, 11, 19, 17, 117, 12, 79, 13, 15, 19, 17, 25, 29, 15, 10, 13, 19, 19, 19, 15, 19, 5, 34, 17, 22, 17, 19, 21, 52, 13, 58, 21, 60, 13, 93, 31, 15, 52, 45, 34, 51, 15, 18, 19, 16, 13, 40, 62, 23, 15, 21, 21, 59, 40, 99, 66, 18, 70, 13, 29, 17, 21, 15, 97, 15, 50, 19, 17, 22, 17, 73, 44, 55, 15, 5

IndexError: list index out of range

In [11]:
print(new_vasp_calls)

[[37, 7, 27, 67, 21, 33, 13, 30, 47, 11], [28, 40, 11, 31, 24, 27, 44, 25, 25, 42], [23, 18, 13, 38, 12, 30, 35, 41, 33, 21], [33, 28, 18, 49, 30, 30, 17, 27, 24, 22], [21, 26, 30, 34, 39, 29, 32, 34, 27, 34], [19, 22, 24, 47, 28, 28, 52, 54, 21, 24], [29, 29, 14, 12, 21, 35, 22, 42, 24, 33], [40, 29, 37, 33, 26, 22, 26, 33, 46, 21], [27, 15, 43, 28, 31, 35, 32, 24, 20, 32], [26, 36, 23, 33, 41, 21, 21, 28, 33, 19], [29, 30, 30, 19, 18, 41, 32, 29, 31, 13], [30, 21, 30, 31, 24, 41, 34, 26, 36, 23], [39, 17, 23, 30, 43, 30, 16, 29, 32, 24], [31, 37, 27, 38, 18, 26, 6, 31, 32, 23], [4, 39, 30, 34, 28, 34, 35, 36, 52, 27], [10, 30, 28, 23, 39, 28, 38, 37, 21, 30], [30, 24, 33, 41, 10, 26, 23, 35, 48, 27], [32, 36, 22, 28, 36, 28, 6, 27, 35], [22, 24, 37, 39, 26, 59, 22, 23, 20, 11], [19, 34, 31, 18, 9, 30, 31, 35, 29, 31], [21, 39, 24, 49, 28, 31, 29, 11, 38, 47], [32, 32, 28, 36, 30, 25, 37, 16, 35, 13], [7, 52, 24, 37, 25, 42, 21, 4, 19, 24], [26, 32, 24, 23, 18, 26, 25, 7, 17, 27], [40

In [12]:
print(similar_clus)

[(0, 9), (2, 1), (5, 0), (6, 1), (6, 2), (7, 9), (8, 2), (10, 3), (10, 4), (11, 3), (12, 3), (13, 0), (14, 8), (18, 2), (19, 0), (19, 7), (22, 2), (22, 9)]


In [13]:
same_clus_almlp = []
same_clus_vasp = []
same_clus_diff = []
for i in range(len(similar_clus)):
    p = similar_clus[i][0]
    q = similar_clus[i][1]
    
    r = new_vasp_calls[p][q] #clus_XXX.log AL
    same_clus_almlp.append(r)
    s = vasp_calls_list[p][q] #results.log DFT
    same_clus_vasp.append(s)
    
    same_clus_diff.append(s-r)
    #print(r, s, s-r)
print(same_clus_almlp)
print(sum(same_clus_almlp))
print(same_clus_vasp)
print(sum(same_clus_vasp))
print(same_clus_diff)
print(sum(same_clus_diff))

[11, 18, 19, 29, 14, 21, 43, 19, 18, 31, 30, 31, 52, 37, 19, 35, 24, 24]
475
[19, 15, 39, 21, 17, 15, 48, 47, 18, 58, 15, 5, 15, 79, 10, 5, 40, 40]
506
[8, -3, 20, -8, 3, -6, 5, 28, 0, 27, -15, -26, -37, 42, -9, -30, 16, 16]
31
